<a href="https://colab.research.google.com/github/Isaac-Gregory/Continual-Learning-Project/blob/data_aug/Continual_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Continual Learning Demonstration

CSCE 5280 - AI for Healthcare and Wearables

Professor: Dr. Mark Albert  

TA: Mr. Riyad Bin Rafiq


---


Students working on the project:

Fahmid Shahriar Iqbal,
Gayaetiri Chalasani,
Isaac Gregory,
Joseph Caldwell

In [4]:
# Imports
import kagglehub
import os
import numpy as np
# import torchvision.transforms as transforms
from torchvision.transforms import v2
import torch
import pandas as pd
from torchvision import datasets, transforms

## Dataset

### Gathering Images and Labels

In [5]:
# Downloading data (NOTE: There are no issues if data is already downloaded)
path = kagglehub.dataset_download("grassknoted/asl-alphabet") + "/asl_alphabet_train/asl_alphabet_train"
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/grassknoted/asl-alphabet/versions/1/asl_alphabet_train/asl_alphabet_train


Creating folder setup for loading into PyTorch later...

In [7]:
# Separating each task's directories
task1_dirs = ["A", "B", "C", "D", "E", "F", "G", "H"]
task2_dirs = ["I", "K", "L", "M", "N", "O", "P", "Q"]
task3_dirs = ["R", "S", "T", "U", "V", "W", "X", "Y"]

# Creating new directories in path for different tasks
# (This helps in loading to pytorch later)
if not os.path.exists(path + "/task1"):
  os.mkdir(path + "/task1")
  os.mkdir(path + "/task2")
  os.mkdir(path + "/task3")

  # Moving directories into respective tasks
  for dir in task1_dirs:
    os.rename(path + "/" + dir, path + "/task1/" + dir)
  for dir in task2_dirs:
    os.rename(path + "/" + dir, path + "/task2/" + dir)
  for dir in task3_dirs:
    os.rename(path + "/" + dir, path + "/task3/" + dir)

Storing file paths and labels into dataframe...

(NOTE: this may prove to be unnecessary now)

In [8]:
task1_list = []
task2_list = []
task3_list = []

# Walking through folders
for dir, _, files in os.walk(path):
  for filename in files:

    # Printing labels and photos
    # print(dirname[-1], filename)

    # Gathering data into dataframes
    # (each letter except J and Z due to movement)
    if dir[-1] in task1_dirs:
      task1_list.append((os.path.join(dir, filename), dir[-1]))
    elif dir[-1] in task2_dirs:
      task2_list.append((os.path.join(dir, filename), dir[-1]))
    elif dir[-1] in task3_dirs:
      task3_list.append((os.path.join(dir, filename), dir[-1]))

task1 = pd.DataFrame(task1_list, columns=['file_path', 'target'])
task2 = pd.DataFrame(task2_list, columns=['file_path', 'target'])
task3 = pd.DataFrame(task3_list, columns=['file_path', 'target'])

Checking the data...

In [9]:
print(np.unique(task1['target']), task1.shape)
print(np.unique(task2['target']), task2.shape)
print(np.unique(task3['target']), task3.shape)

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H'] (24000, 2)
['I' 'K' 'L' 'M' 'N' 'O' 'P' 'Q'] (24000, 2)
['R' 'S' 'T' 'U' 'V' 'W' 'X' 'Y'] (24000, 2)


In [ ]:
task1.head()

In [ ]:
task2.head()

In [ ]:
task3.head()

In [ ]:
print(task1.groupby('target').count())
print(task2.groupby('target').count())
print(task3.groupby('target').count())

### EDA

### Data Augmentation

In [10]:
torch.manual_seed(1)

In [ ]:
# # Calculate mean and std
# vals = [[],[],[]]
# mean = []
# std = []
# temp_dataset = datasets.ImageFolder(path + "/task1", transform=transforms.ToTensor())
# for images, _ in temp_dataset:
#     for j in range(3):  # Assuming 3 channels (RGB)
#         channel = images[j, :, :]
#         vals[j].append(channel)

# for i in range(3):
#     # vals[i] = torch.stack(vals[i])
#     mean.append(np.mean(vals[i]))
#     std.append(np.std(vals[i]))

In [ ]:
# print(mean)
# print(std)

In [ ]:
# Creating image transformations/augmentations
transforms = v2.Compose([
    v2.RandomResizedCrop(size=(200, 200), antialias=True),
    v2.RandomRotation(degrees=90, expand=False),
    v2.ColorJitter(),
    v2.GaussianBlur(),
    v2.Normalize(),
    v2.ToDtype(torch.float32, scale=True),
    v2.ToTensor(),
])

In [ ]:
dataset = datasets.ImageFolder(path, transform=transforms)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)